In [3]:
import sys
print(sys.executable)


/usr/local/python/3.10.13/bin/python3


In [4]:
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
model_name = 'multi-qa-distilbert-cos-v1'

In [ ]:

embedding_model = SentenceTransformer(model_name)

In [ ]:
user_question = "I just discovered the course. Can I still join it?"

In [ ]:
model.encode(")